In [1]:
%run "nb_spatial_analysis_functions"

StatementMeta(, 2dffe830-4473-44e4-afd1-f86c98ede308, 9, Finished, Available, Finished)

In [2]:
# Load the KML file (specify driver explicitly)
gdf = gpd.read_file("/lakehouse/default/Files/force kmls/cumbria.kml", driver='KML')

# Explode MultiPolygons into individual Polygons
gdf = gdf.explode(index_parts=False)

# Apply h3pandas polyfill
hex_gdf = gdf.h3.polyfill(resolution=9)

exploded_df = hexploded_df = hex_gdf.explode('h3_polyfill', ignore_index=True).rename(columns={'h3_polyfill':'h3_index_9'})
h3_list = exploded_df[['h3_index_9']]

df_h3 = spark.createDataFrame(h3_list) 
display(df_h3)

StatementMeta(, 2dffe830-4473-44e4-afd1-f86c98ede308, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3665d7cd-b696-449b-a6b7-a9853624d689)

In [3]:
get_parent_h3_index_udf = udf(get_parent_h3_index, StringType())
h3_to_wkt_polygon_udf = udf(h3_to_wkt_polygon,StringType())

df_h3 = df_h3.withColumn("h3_index_8", get_parent_h3_index_udf(df_h3["h3_index_9"],lit(8)))
df_h3 = df_h3.withColumn("h3_index_7", get_parent_h3_index_udf(df_h3["h3_index_9"],lit(7)))
df_h3 = df_h3.withColumn("h3_index_6", get_parent_h3_index_udf(df_h3["h3_index_9"],lit(6)))

df_h3 = df_h3.withColumn("h3_9_wkt", h3_to_wkt_polygon_udf(df_h3["h3_index_9"]))
df_h3 = df_h3.withColumn("h3_8_wkt", h3_to_wkt_polygon_udf(df_h3["h3_index_8"]))
df_h3 = df_h3.withColumn("h3_7_wkt", h3_to_wkt_polygon_udf(df_h3["h3_index_7"]))
df_h3 = df_h3.withColumn("h3_6_wkt", h3_to_wkt_polygon_udf(df_h3["h3_index_6"]))

StatementMeta(, 2dffe830-4473-44e4-afd1-f86c98ede308, 11, Finished, Available, Finished)

In [15]:
display(df_h3)

StatementMeta(, 10ee7411-1585-471c-a119-ca2101ef1b18, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f19da035-8981-43c5-af80-dcf570428567)

In [10]:
df_h3.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable("dim_asb_h3")

StatementMeta(, 2dffe830-4473-44e4-afd1-f86c98ede308, 18, Finished, Available, Finished)